# Boundary vector score (BVS) - datajoint integration

Datajoint integration of bvs calculation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Set up basics
import sys, os
import numpy as np 
from collections import OrderedDict
# Make plots pretty 
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style='white')
%config InlineBackend.figure_format = 'retina'

import datajoint as dj

In [3]:
# Load base schema
schema = dj.schema(dj.config['dj_imaging.database'])
schema.spawn_missing_classes()

Connecting horsto@kavlidatajoint02.it.ntnu.no:3306


### Schema components

In [4]:
# Load personal schema 
borderscore_schema = dj.schema('user_horsto_borderscore')
borderscore_schema.spawn_missing_classes()

In [70]:
borderscore_schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__b_v_field,0d02e89da1fea86dc6278941400fb40b,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,3242,1067,2020-08-30 17:37:03
__b_v_field,0eff9e74301f41faeb9fab1309a6d4ca,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,2721,1061,2020-08-30 17:37:03
__b_v_field,32d88d918265207b882272a619b1e465,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,3064,1065,2020-08-30 17:37:03
__b_v_field,5cfacbfa6ce5d55862115b1243ccd466,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,2557,1059,2020-08-30 17:37:03
__b_v_field,7527d74634e815fd0a21f240485765c4,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,3418,1069,2020-08-30 17:37:03
__b_v_field,e8132e4a2cccd3e54c452707c31ce14a,reserved,=BLOB=,,=BLOB=,horsto@10.212.25.16,jupyterstack,2888,1063,2020-08-30 17:37:03


In [71]:
BVField().progress()

BVField              Completed 3472 of 180392 (1.9%)   2020-08-30 17:37:09


(176920, 180392)

In [69]:
BVScore()

metasession_name Meta session name (hash),session_order Order of session within meta sessions (zero index!),"session_name Session name: Hash of animal_id, datasource_id, timestamp and combined 'yes'/'no' label",signal_dataset 16 character hash,cell_id Cell ID,channel the channel that this trace comes from (ROI masks are always primary),"spike_filter_id Parameter set ID, starting with A",tracking_dataset 16 character hash,"trackingparams_id Parameter set ID, starting with A","s_t_params_id Parameter set ID, starting with A",signal_type Signal type (Fluorescence vs. Spikes),"map_params_id Parameter set ID, starting with A","field_params_id Parameter set ID, starting with A","bvfield_params_id Parameter set ID, starting with A",bv_field_dect_method Specifies how fields were extracted,bvs Boundary vector score (BVS),orientation
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,1,primary,A,d83d77c1591c8c9f,A,A,df_f,A,A,A,opexebo,0.6648185483870968,horizontal
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,1,primary,A,d83d77c1591c8c9f,A,A,df_f,A,A,A,bvs,0.4747983870967742,vertical
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,1,primary,A,d83d77c1591c8c9f,A,A,df_f,B,A,A,opexebo,0.5346153846153846,vertical
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,1,primary,A,d83d77c1591c8c9f,A,A,df_f,B,A,A,bvs,0.4461538461538461,horizontal
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,1,primary,A,d83d77c1591c8c9f,A,A,spikes,A,A,A,opexebo,0.8235887096774194,horizontal
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,1,primary,A,d83d77c1591c8c9f,A,A,spikes,A,A,A,bvs,0.3810483870967742,horizontal
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,1,primary,A,d83d77c1591c8c9f,A,A,spikes,B,A,A,opexebo,0.4076923076923077,horizontal
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,1,primary,A,d83d77c1591c8c9f,A,A,spikes,B,A,A,bvs,0.40384615384615385,horizontal
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,20,primary,A,d83d77c1591c8c9f,A,A,df_f,A,A,A,opexebo,0.6622983870967742,vertical
019bb8938659cfee,0,218598b321b72d27,62325d8521fc7d42,20,primary,A,d83d77c1591c8c9f,A,A,df_f,A,A,A,bvs,0.40120967741935487,None
